In [2]:
%pwd

'c:\\Users\\kgn\\OneDrive - PowerSchool\\PowerSchool\\Release Script\\Python\\Data Scientist\\LLMScratch\\notebook'

In [3]:
import os
os.chdir("../")
%pwd

'c:\\Users\\kgn\\OneDrive - PowerSchool\\PowerSchool\\Release Script\\Python\\Data Scientist\\LLMScratch'

In [4]:
import json
file_path = "notebook/modules/instruction-data.json"

with open(file_path, "r") as file:
    data = json.load(file)
print("Number of entries:", len(data))

Number of entries: 1100


In [5]:
train_portion = int(len(data) * 0.85)  # 85% for training
test_portion = int(len(data) * 0.15)    # 15% for testing

train_data = data[:train_portion]
test_data = data[train_portion:]

In [5]:
print("Training set length:", len(train_data))
print("Test set length:", len(test_data))

Training set length: 935
Test set length: 165


In [6]:
with open("notebook/modules/train.json", "w") as json_file:
    json.dump(train_data, json_file, indent=4)
    
with open("notebook/modules/test.json", "w") as json_file:
    json.dump(test_data, json_file, indent=4)

In [ ]:
!litgpt finetune_lora microsoft/phi-2 \
--data JSON \
--data.val_split_fraction 0.1 \
--data.json_path notebook/modules/train.json \
--train.epochs 3 \
--train.log_interval 100

In [9]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

print(test_data[0],"\n\n")
print(format_input(test_data[0]))

{'instruction': 'Rewrite the sentence using a simile.', 'input': 'The car is very fast.', 'output': 'The car is as fast as lightning.'} 


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Rewrite the sentence using a simile.

### Input:
The car is very fast.


In [10]:
from litgpt import LLM
from tqdm import tqdm

llm = LLM.load("microsoft/phi-2")
response = llm.generate(format_input(test_data[0]))

c:\Users\kgn\OneDrive - PowerSchool\PowerSchool\Release Script\Python\Data Scientist\LLMScratch\.llms3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\kgn\OneDrive - PowerSchool\PowerSchool\Release Script\Python\Data Scientist\LLMScratch\.llms3.10\lib\site-packages\litgpt\utils.py:559: UserWarning: The file size of checkpoints\microsoft\phi-2\lit_model.pth is over 4.2 GB. Using a model with more than 1B parameters on a CPU can be slow, it is recommended to switch to a GPU.
  warnings.warn(


In [ ]:
from tqdm import tqdm

for i in tqdm(range(len(test_data))):
    response = llm.generate(test_data[i])
    test_data[i]["base_model"] = response

  0%|          | 0/165 [00:00<?, ?it/s]

Using LORA

In [ ]:
from litgpt import LLM

del llm
llm2 = LLM.load("out/finetune/lora/final/")

In [ ]:
from tqdm import tqdm

for i in tqdm(range(len(test_data))):
    response = llm2.generate(test_data[i])
    test_data[i]["finetuned_model"] = response